In [ ]:
#configuración en google colab de spark y pyspark
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#configuración en google colab
#instalar java y spark
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.5.2/spark-3.5.2-bin-hadoop3.tgz
!tar xf spark-3.5.2-bin-hadoop3.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"
import findspark
findspark.init()

In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

In [ ]:
df=spark.createDataFrame([(1,'I really liked this movie'),
                         (2,'I would recommend this movie to my friends'),
                         (3,'movie was alright but acting was horrible'),
                         (4,'I am never watching that movie ever again')],
                        ['user_id','review'])

In [ ]:
df.show(5,False)

In [ ]:
# Tokenization

In [ ]:
from pyspark.ml.feature import Tokenizer

In [ ]:
tokenization=Tokenizer(inputCol='review',outputCol='tokens')

In [ ]:
tokenized_df=tokenization.transform(df)

In [ ]:
tokenized_df.show(4,False)

In [ ]:
# stopwords removal 

In [ ]:
from pyspark.ml.feature import StopWordsRemover

In [ ]:
stopword_removal=StopWordsRemover(inputCol='tokens',outputCol='refined_tokens')

In [ ]:
refined_df=stopword_removal.transform(tokenized_df)

In [ ]:
refined_df.select(['user_id','tokens','refined_tokens']).show(10,False)

In [ ]:
# Count Vectorizer

In [ ]:
from pyspark.ml.feature import CountVectorizer

In [ ]:
count_vec=CountVectorizer(inputCol='refined_tokens',outputCol='features')

In [ ]:
cv_df=count_vec.fit(refined_df).transform(refined_df)

In [ ]:
cv_df.select(['user_id','refined_tokens','features']).show(4,False)

In [ ]:
count_vec.fit(refined_df).vocabulary

In [ ]:
#Tf-idf

In [ ]:
from pyspark.ml.feature import HashingTF,IDF

In [ ]:
hashing_vec=HashingTF(inputCol='refined_tokens',outputCol='tf_features',numFeatures=100)

In [ ]:
hashing_df=hashing_vec.transform(refined_df)

In [ ]:
hashing_df.select(['user_id','refined_tokens','tf_features']).show(4,False)

In [ ]:
tf_idf_vec=IDF(inputCol='tf_features',outputCol='tf_idf_features')

In [ ]:
tf_idf_df=tf_idf_vec.fit(hashing_df).transform(hashing_df)

In [ ]:
tf_idf_df.select(['user_id','tf_idf_features']).show(4,False)

In [ ]:
# Classification 

In [ ]:
# en google colab:
# text_df=spark.read.csv('/content/gdrive/MyDrive/st1800github/datasets/movie_reviews.csv',inferSchema=True,header=True,sep=',')

# local:
text_df=spark.read.csv('../datasets/movie_reviews.csv',inferSchema=True,header=True,sep=',')

In [ ]:
text_df.printSchema()

In [ ]:
text_df.count()

In [ ]:
from pyspark.sql.functions import rand 

In [ ]:
text_df.orderBy(rand()).show(10,False)

In [ ]:
text_df=text_df.filter(((text_df.Sentiment =='1') | (text_df.Sentiment =='0')))

In [ ]:
text_df.count()

In [ ]:
text_df.groupBy('Sentiment').count().show()

In [ ]:
text_df.printSchema()

In [ ]:
text_df = text_df.withColumn("Label", text_df.Sentiment.cast('float')).drop('Sentiment')

In [ ]:
text_df.orderBy(rand()).show(10,False)

In [ ]:
text_df.groupBy('label').count().show()

In [ ]:
# Add length to the dataframe
from pyspark.sql.functions import length

In [ ]:
text_df=text_df.withColumn('length',length(text_df['Review']))

In [ ]:
text_df.orderBy(rand()).show(10,False)

In [ ]:
text_df.groupBy('Label').agg({'Length':'mean'}).show()

In [ ]:
# Data Cleaning

In [ ]:
tokenization=Tokenizer(inputCol='Review',outputCol='tokens')

In [ ]:
tokenized_df=tokenization.transform(text_df)

In [ ]:
tokenized_df.show()

In [ ]:
stopword_removal=StopWordsRemover(inputCol='tokens',outputCol='refined_tokens')

In [ ]:
refined_text_df=stopword_removal.transform(tokenized_df)

In [ ]:
refined_text_df.show()

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import *

In [ ]:
len_udf = udf(lambda s: len(s), IntegerType())

refined_text_df = refined_text_df.withColumn("token_count", len_udf(col('refined_tokens')))


In [ ]:
refined_text_df.orderBy(rand()).show(10)

In [ ]:
count_vec=CountVectorizer(inputCol='refined_tokens',outputCol='features')

In [ ]:
cv_text_df=count_vec.fit(refined_text_df).transform(refined_text_df)

In [ ]:
cv_text_df.select(['refined_tokens','token_count','features','Label']).show(10)

In [ ]:
#select data for building model
model_text_df=cv_text_df.select(['features','token_count','Label'])

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
df_assembler = VectorAssembler(inputCols=['features','token_count'],outputCol='features_vec')
model_text_df = df_assembler.transform(model_text_df)

In [ ]:
model_text_df.printSchema()

In [ ]:
from pyspark.ml.classification import LogisticRegression

In [ ]:
#split the data 
training_df,test_df=model_text_df.randomSplit([0.75,0.25])

In [ ]:
training_df.groupBy('Label').count().show()

In [ ]:
test_df.groupBy('Label').count().show()

In [ ]:
log_reg=LogisticRegression(featuresCol='features_vec',labelCol='Label').fit(training_df)

In [ ]:
results=log_reg.evaluate(test_df).predictions

In [ ]:
results.select(['Label','probability','prediction']).show()

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator


In [ ]:
#confusion matrix
true_postives = results[(results.Label == 1) & (results.prediction == 1)].count()
true_negatives = results[(results.Label == 0) & (results.prediction == 0)].count()
false_positives = results[(results.Label == 0) & (results.prediction == 1)].count()
false_negatives = results[(results.Label == 1) & (results.prediction == 0)].count()

In [ ]:
recall = float(true_postives)/(true_postives + false_negatives)
print(recall)

In [ ]:
precision = float(true_postives) / (true_postives + false_positives)
print(precision)

In [ ]:
accuracy=float((true_postives+true_negatives) /(results.count()))
print(accuracy)